# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [44]:
!pip install ultralytics

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [51]:
import cv2
from ultralytics import YOLO
model = YOLO('/content/YOLO_Model.pt')
model.info()

Model summary: 365 layers, 68,156,460 parameters, 0 gradients, 258.1 GFLOPs


(365, 68156460, 0, 258.13867519999997)

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [52]:
cap = cv2.VideoCapture('/content/YOLO_Video.mp4')

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [53]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [54]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_path = '/content/YOLO_Video.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [63]:
import cv2

fourcc = cv2.imread('//content/YOLO_Video-2.mp4')

results = model(fourcc)

for result in results:
  for box in result.boxes:
    x1, x2, y1, y2 = map(int, box.xyxy[0])
    label = result.names[int(box.cls)]
    confidence = box.conf[0]

    cv2.rectangle(fourcc, (x1, y1), (x2,y2), (0,255,0),2)
    cv2.putText(fourcc, f'{label} {confidence:.2f}', (x1, y1 -10), cv2.FONT_HERSHEY_COMPLEX, 0.5,  (0,255,0),2)


WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 640x480 12 cars, 3148.9ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 384x640 1 car, 2450.6ms
Speed: 4.0ms preprocess, 2799.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


In [58]:
roi_top = (width // 4, 0, 3 * width // 4, height // 8)
roi_bottom = (width // 5, 7 * height // 7, 3 * width // 3, height)
roi_right = (6 * width // 8, height // 4, width, 3 * height // 4)
roi_left = (0, height // 4, width // 8, 3 * height // 4)

In [67]:
count_top = 0
count_bottom = 0
count_right = 0
count_left = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


    results = model(frame)


    cv2.rectangle(frame, (roi_top[0], roi_top[1]), (roi_top[2], roi_top[3]), (0, 255, 0), 2)
    cv2.rectangle(frame, (roi_bottom[0], roi_bottom[1]), (roi_bottom[2], roi_bottom[3]), (0, 255, 255), 2)
    cv2.rectangle(frame, (roi_right[0], roi_right[1]), (roi_right[2], roi_right[3]), (255, 0, 255), 2)
    cv2.rectangle(frame, (roi_left[0], roi_left[1]), (roi_left[2], roi_left[3]), (255, 0, 0), 2)

    for detection in results:
        for obj in detection.boxes:
            x1, y1, x2, y2 = map(int, obj.xyxy[0])
            conf = obj.conf[0]
            cls = int(obj.cls[0])


            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            label = f"{cls}, {conf:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)


            if roi_top[0] < x1 < roi_top[2] and roi_top[1] < y1 < roi_top[3]:
                count_top += 1
            if roi_bottom[0] < x1 < roi_bottom[2] and roi_bottom[1] < y1 < roi_bottom[3]:
                count_bottom += 1
            if roi_right[0] < x1 < roi_right[2] and roi_right[1] < y1 < roi_right[3]:
                count_right += 1
            if roi_left[0] < x1 < roi_left[2] and roi_left[1] < y1 < roi_left[3]:
                count_left += 1


    cv2.putText(frame, f"Top: {count_top}", (roi_top[0] + 10, roi_top[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
    cv2.putText(frame, f"Bottom: {count_bottom}", (roi_bottom[0] + 10, roi_bottom[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
    cv2.putText(frame, f"Right: {count_right}", (roi_right[0] + 10, roi_right[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
    cv2.putText(frame, f"Left: {count_left}", (roi_left[0] + 10, roi_left[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)


    cv2.putText(frame, 'Alanoud', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)


    out.write(frame)

In [68]:
cap.release()
out.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video